Load in Data (Regular Season (Detailed), Tournament Seeds, Tournament Game Results).  
Updated to filter out games before 2003 from the tournament datasets, since the newly added regular season detailed dataset starts at 2003. 

In [31]:
import pandas as pd
import numpy as np
reg_df = pd.read_csv('../data/raw/MRegularSeasonDetailedResults.csv')
seed_df = pd.read_csv('../data/raw/MNCAATourneySeeds.csv')
tourney_df = pd.read_csv('../data/raw/MNCAATourneyCompactResults.csv')
seed_df = seed_df[seed_df['Season'] >= 2003]
tourney_df = tourney_df[tourney_df['Season'] >= 2003]

Collect the data needed for future feature calculations. The dataset is organized into a game per row, with data for the winning team all beginning with W, and data for the losing team beginning with L. The data is gathered and combined for when a team is both a winner and a loser for every game throughout the entire season. 

In [32]:
# Statistics when the team is the winner, ws = winner stats
ws_df = reg_df.groupby(['Season', 'WTeamID']).agg({
    'WScore': 'sum', 
    'LScore': 'sum', 
    'WFGA': 'sum', 
    'WOR': 'sum', 
    'WTO': 'sum', 
    'WFTA': 'sum', 
    'WFGM': 'sum', 
    'WFGM3': 'sum', 
    'WAst': 'sum'}).reset_index()
ws_df.rename(columns={
    'WTeamID': 'TeamID', 
    'WScore': 'PS', # Points Scored
    'LScore': 'PA', # Points Allowed
    'WFGA': 'FGA', 
    'WOR': 'OR', 
    'WTO': 'TO', 
    'WFTA': 'FTA', 
    'WFGM': 'FGM', 
    'WFGM3': 'FGM3',
    'WAst': 'AST'}, inplace = True)

# Do the same for when the team is the loser, ls = loser stats
ls_df = reg_df.groupby(['Season', 'LTeamID']).agg({
    'WScore': 'sum', 
    'LScore': 'sum', 
    'LFGA': 'sum', 
    'LOR': 'sum', 
    'LTO': 'sum', 
    'LFTA': 'sum', 
    'LFGM': 'sum', 
    'LFGM3': 'sum', 
    'LAst': 'sum'}).reset_index()
ls_df.rename(columns={
    'LTeamID': 'TeamID', 
    'WScore': 'PA', 
    'LScore': 'PS', 
    'LFGA': 'FGA', 
    'LOR': 'OR', 
    'LTO': 'TO', 
    'LFTA': 'FTA', 
    'LFGM': 'FGM', 
    'LFGM3': 'FGM3', 
    'LAst': 'AST'}, inplace = True)

# Combine into a table where each record will include a team's overall statistics for the season, rss = regular season stats
rss_df = pd.concat([ws_df, ls_df]).groupby(['Season', 'TeamID']).sum().reset_index()

Perform calculations from the regular season data  
Calculate Possessions (Someone should tally these at the games, they count everything else):  
Possession = (FGA - OR) + TO + (0.44 * FTA)  

Calculate offensive, defensive, and net rating, and assists to turnover ratio:  
Offensive Rating (ORTG) = (PS / POSS) * 100  
Defensive Rating (DRTG) = (PA / POSS) * 100  
Net Rating (NRTG) = ORTG - DRTG  
Assist to Turnover Ratio = AST / TO

Calculate shooting metrics:  
Effective Field Goal Percentage (eFG) = (FGM + 0.5 * 3PM) / FGA  
True Shooting Percentage (TS) = PS / 2 * (FGA + 0.475 * FTA)

In [33]:
rss_df['POSS'] = (rss_df['FGA'] - rss_df['OR']) + rss_df['TO'] + (0.44 * rss_df['FTA'])
rss_df['ORTG'] = (rss_df['PS'] / rss_df['POSS']) * 100
rss_df['DRTG'] = (rss_df['PA'] / rss_df['POSS']) * 100
rss_df['NRTG'] = (rss_df['ORTG'] - rss_df['DRTG'])
rss_df['ASTO'] = rss_df['AST'] / rss_df['TO'] # ASTO = assists / turnovers
rss_df['EFG'] = (rss_df['FGM'] + 0.5 * rss_df['FGM3']) / rss_df['FGA']
rss_df['TS'] = rss_df['PS'] / (2 * (rss_df['FGA'] + 0.475 * rss_df['FTA']))

The model will be predict based on tournament matchups and each team's regular season stats (built above). An intial table just representing a tournament matchup, the seed and seed difference of each team, and the outcome will first be built, and then each team's stats will be added on. 
Since the tournament dataset is organized the same as the regular season, with W and L fields for winning and losing team's, Team1 is assigned to the team with a greater ID number to avoid the model learning team1 or team2 always wins. 

In [34]:
# Gather tournament data, and create matchup dataframe where the regular season stats will be attached
tournament_data = {
    'Season': tourney_df['Season'],
    'Team1': np.maximum(tourney_df['WTeamID'], tourney_df['LTeamID']),
    'Team2': np.minimum(tourney_df['WTeamID'], tourney_df['LTeamID']),
    'Winner': (tourney_df['WTeamID'] == np.maximum(tourney_df['WTeamID'], tourney_df['LTeamID'])).astype(int),
}
matchup_df = pd.DataFrame(tournament_data)

# Modify seed table to include a column of the seed number (rather than original format that included a location character)
seed_df['SeedNum'] = seed_df['Seed'].str.extract(r'(\d+)').astype(int)

# Attach team1's seed
matchup_df = matchup_df.merge(seed_df, how = 'left', left_on = ('Season', 'Team1'), right_on = ('Season', 'TeamID'))
matchup_df.drop(columns=['TeamID', 'Seed'], inplace = True)

# Attach team2's seed
matchup_df = matchup_df.merge(seed_df, how = 'left', left_on = ('Season', 'Team2'), right_on = ('Season', 'TeamID'), suffixes = ('', '_2'))
matchup_df.drop(columns=['TeamID', 'Seed'], inplace = True)

In [35]:
# Attach Team1's stats
matchup_df = matchup_df.merge(rss_df, how = 'left', left_on = ('Season', 'Team1'), right_on = ('Season', 'TeamID'))
matchup_df.drop(columns=['TeamID'], inplace = True)

In [36]:
# Attach Team2's stats
matchup_df = matchup_df.merge(rss_df, how = 'left', left_on = ('Season', 'Team2'), right_on = ('Season', 'TeamID'), suffixes = ('', '_2'))
matchup_df.drop(columns=['TeamID'], inplace = True)

Perform difference calculations for the features

In [45]:
# Compute the differences
matchup_df['SeedDiff'] = matchup_df['SeedNum_2'] - matchup_df['SeedNum']
matchup_df['NDiff'] = matchup_df['NRTG'] - matchup_df['NRTG_2']
matchup_df['ATDiff'] = matchup_df['ASTO'] - matchup_df['ASTO_2']
matchup_df['EFGDiff'] = matchup_df['EFG'] - matchup_df['EFG_2']
matchup_df['TSDiff'] = matchup_df['TS'] - matchup_df['TS_2']

Take everything from the matchup dataframe that will be used as features in the model

In [51]:
# Create the final feature table from everything that's been calculated
feature_data = {
    'Season': matchup_df['Season'], 
    'NDiff': matchup_df['NDiff'],
    'ATDiff': matchup_df['ATDiff'],
    'EFGDiff': matchup_df['EFGDiff'],
    'TSDiff': matchup_df['TSDiff'],
    'SeedDiff': matchup_df['SeedDiff'],
    'Winner': matchup_df['Winner']
}
features_df = pd.DataFrame(feature_data)